In [ ]:
import random

In [1]:
class PuzzleState:
    def __init__(self, board, parent=None, move=None, g=0):
        self.board = board  # The board state as a tuple
        self.parent = parent  # The parent state
        self.move = move  # The move that led to this state
        self.g = g  # Cost from the start state
        self.h = self.heuristic()  # Heuristic value
        self.f = self.h  # Total cost for hill climbing (only heuristic value matters)

    def heuristic(self):
        """Calculate the Manhattan Distance for the heuristic function."""
        distance = 0
        for i in range(9):
            value = self.board[i]
            if value != 0:  # Skip the empty space
                target_pos = value - 1
                target_row, target_col = divmod(target_pos, 3)
                current_row, current_col = divmod(i, 3)
                distance += abs(target_row - current_row) + abs(target_col - current_col)
        return distance

    def __lt__(self, other):
        """Allow comparison of states based on the heuristic value."""
        return self.f < other.f

class Puzzle:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state

    def goal_test(self, state):
        """Check if the current state matches the goal state."""
        return state == self.goal_state

    def get_successors(self, state):
        """Generate all possible successor states by moving the empty space."""
        size = 3  # 3x3 puzzle
        successors = []
        empty_pos = state.board.index(0)  # Find the position of the empty space
        x, y = divmod(empty_pos, size)  # Get row, column of empty space
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < size and 0 <= ny < size:  # Check if within bounds
                new_pos = nx * size + ny
                new_board = list(state.board)
                # Swap the empty space with the adjacent tile
                new_board[empty_pos], new_board[new_pos] = new_board[new_pos], new_board[empty_pos]
                successors.append(PuzzleState(tuple(new_board), state, (x, y)))
        
        return successors

    def steepest_ascent_hill_climbing(self):
        """Solve the 8-puzzle problem using steepest ascent hill climbing."""
        current_state = PuzzleState(self.initial_state)

        while True:
            # Check if the current state is the goal state
            if self.goal_test(current_state.board):
                return self.reconstruct_path(current_state)

            # Generate successors and find the best successor
            successors = self.get_successors(current_state)
            best_successor = min(successors, key=lambda s: s.h)

            # If no improvement, stop (local maxima or goal reached)
            if best_successor.h >= current_state.h:
                break

            current_state = best_successor

        return None  # No solution found

    def reconstruct_path(self, state):
        """Reconstruct the path from the goal state to the initial state."""
        path = []
        while state is not None:
            path.append(state.board)
            state = state.parent
        path.reverse()  # Reverse to get the path from start to goal
        return path

    def print_solution(self, path):
        """Print the solution path."""
        if path is None:
            print("No solution found or stuck in a local maximum.")
        else:
            for step in path:
                self.print_board(step)

    def print_board(self, board):
        """Print the board in a human-readable format."""
        for i in range(0, 9, 3):
            print(board[i:i+3])
        print()

In [2]:
initial_state = (1, 2, 3, 4, 5, 6, 0, 7, 8)  # An example unsolved puzzle
goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 0)  # Goal state

puzzle = Puzzle(initial_state, goal_state)
path = puzzle.steepest_ascent_hill_climbing()
puzzle.print_solution(path)

(1, 2, 3)
(4, 5, 6)
(0, 7, 8)

(1, 2, 3)
(4, 5, 6)
(7, 0, 8)

(1, 2, 3)
(4, 5, 6)
(7, 8, 0)

